# Dynamical Unit

在房室与HH模型之上，``BrainCell`` 通过 ``DiffEqState`` 和 ``DiffEqModule`` 实现神经元动态的数学描述。通过``DiffEqState`` 和 ``DiffEqModule``，``BrainCell`` 将电生理学过程转化为严格的微分方程系统，从而连接微观生物机制与宏观系统表现。


## 一、DiffEqState

### 作用
``DiffEqState`` 是管理系统状态变量的核心组件，负责统一存储、初始化和访问神经元动态中的所有状态变量。它为微分方程系统提供了结构化的状态容器，确保状态变量的一致性和可追踪性，同时支持批量处理和并行计算，适配从单神经元到网络的多尺度模拟。

### 实现
```python
import braincell
import brainunit as u
import brainstate

# 定义神经元模型的状态变量：膜电位、门控变量、钙浓度
state = braincell.DiffEqState(
    variables={
        'V': brainstate.init.Constant(-65. * u.mV),  # 膜电位
        'm': brainstate.init.Constant(0.05),         # Na⁺通道激活门
        'h': brainstate.init.Constant(0.6),          # Na⁺通道失活门
        'n': brainstate.init.Constant(0.32),         # K⁺通道激活门
        'Ca': brainstate.init.Constant(5e-5 * u.mM)  # 钙离子浓度
    },
    size=1
)

# 访问状态变量
print(f"初始膜电位: {state.V.value}")
print(f"初始钙浓度: {state.Ca.value}")

# 更新状态变量
state.V.value = -50. * u.mV
print(f"更新后膜电位: {state.V.value}")
```


## 二、DiffEqModule

### 作用
``DiffEqModule`` 用于定义状态变量的演化规则，即微分方程组 $ \frac{dx}{dt} = f(x, t) $。它接收 ``DiffEqState`` 中的当前状态，结合外部输入计算各变量的时间导数，是实现神经元动态数学描述的核心模块。通过 ``DiffEqModule``，可以灵活定义从简单RC电路到复杂多离子系统的动力学方程，将电生理机制转化为可求解的数学表达式，为数值积分提供基础。

### 实现
```python
import braincell
import brainunit as u
import brainstate
import jax.numpy as jnp

# 定义基于HH模型的微分方程模块
class HHDiffEq(braincell.DiffEqModule):
    def __init__(self, Cm=1.0 * u.uF / u.cm**2, area=5000 * u.um**2):
        super().__init__()
        self.Cm = Cm * area  # 总膜电容
        self.g_Na = 120. * u.mS / u.cm**2 * area  # 钠通道总电导
        self.g_K = 36. * u.mS / u.cm**2 * area    # 钾通道总电导
        self.g_L = 0.3 * u.mS / u.cm**2 * area    # 漏通道总电导
        self.E_Na = 50. * u.mV                    # 钠反转电位
        self.E_K = -77. * u.mV                    # 钾反转电位
        self.E_L = -54.4 * u.mV                   # 漏电流反转电位

    def derivative(self, state, t, I_input=0. * u.nA):
        # 从状态变量中提取当前值
        V, m, h, n = state.V.value, state.m.value, state.h.value, state.n.value

        # 计算钠通道门控变量的时间导数
        alpha_m = 0.1 * (V + 40) / (1 - jnp.exp(-(V + 40) / 10))
        beta_m = 4.0 * jnp.exp(-(V + 65) / 18)
        dmdt = (alpha_m * (1 - m) - beta_m * m) / u.ms

        alpha_h = 0.07 * jnp.exp(-(V + 65) / 20)
        beta_h = 1 / (1 + jnp.exp(-(V + 35) / 10))
        dhdt = (alpha_h * (1 - h) - beta_h * h) / u.ms

        alpha_n = 0.01 * (V + 55) / (1 - jnp.exp(-(V + 55) / 10))
        beta_n = 0.125 * jnp.exp(-(V + 65) / 80)
        dndt = (alpha_n * (1 - n) - beta_n * n) / u.ms

        # 计算离子电流
        I_Na = self.g_Na * m**3 * h * (V - self.E_Na)
        I_K = self.g_K * n**4 * (V - self.E_K)
        I_L = self.g_L * (V - self.E_L)

        # 计算膜电位的时间导数
        dVdt = (I_input - I_Na - I_K - I_L) / self.Cm

        # 返回所有状态变量的导数
        return {
            'V': dVdt,
            'm': dmdt,
            'h': dhdt,
            'n': dndt
        }

# 初始化状态变量
state = braincell.DiffEqState(
    variables={
        'V': brainstate.init.Constant(-65. * u.mV),
        'm': brainstate.init.Constant(0.05),
        'h': brainstate.init.Constant(0.6),
        'n': brainstate.init.Constant(0.32)
    },
    size=1
)

# 创建微分方程模块实例
hh_module = HHDiffEq()

# 计算某一时刻的导数
derivatives = hh_module.derivative(state, t=0. * u.ms, I_input=10. * u.nA)
print(f"膜电位变化率: {derivatives['V']}")
print(f"钠通道激活门变化率: {derivatives['m']}")
```


## 三、关系

### 层级依赖
- ``DiffEqState`` 是 ``DiffEqModule`` 的输入基础。``DiffEqModule`` 必须依赖 ``DiffEqState`` 提供的状态变量才能计算导数。
- ``DiffEqState`` 和 ``DiffEqModule`` 共同构成微分方程系统。``DiffEqState`` 定义系统状态，``DiffEqModule`` 定义状态变化，二者结合形成完整的微分方程描述，是数值积分求解的前提。

通过这种分工，``Dynamical Unit`` 为神经模型提供了严格的数学基础，既保证生物真实性，又便于利用数值方法高效求解，是连接基础元件与复杂系统的关键层级。

在此基础上，我们可以进行积分求解的工作。``braincell`` 内置了丰富的[积分器](https://braincell.readthedocs.io/latest/apis/integration.html)，你可以根据需要选择合适的数值积分方法。